# Lab 9
This is the jupyter notebook for lab 9. First we will import some libraries and functions. 

In [ ]:
from numpy import genfromtxt
import numpy as np
from netCDF4 import MFDataset 
from netCDF4 import Dataset 
import matplotlib
matplotlib.use('Agg')
import matplotlib.pylab as plt
from datetime import datetime
from netCDF4 import num2date
from netCDF4 import date2index
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
from metpy.units import units
import xarray as xr
import cartopy.util as cutil
%matplotlib inline

Our next setep is to read in the netCDF files. 

In [ ]:
ds = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/air.mon.mean.nc')
ds2 = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc')
ds3 = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc')
ds4 = xr.open_dataset('https://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc')


Provide the pressure level and month you wish to plot. Also provide the month as a string, so our caption is attractive. 

In [ ]:
plevel = 850
month = 1
monthlabel = 'January'
tm = int(month)-1

Now we extract the data desired. 

In [ ]:
# Grab lat/lon values
lats = ds.lat.data
lons = ds.lon.data

#subtract 1 from tm because the first observation in Python is observation 0

temp = ds.air.sel(level=plevel)[tm::12,:,:].values
hght = ds2.hgt.sel(level=plevel)[tm::12,:,:].values
uwnd = np.asarray(ds3.uwnd.sel(level=plevel).values[tm::12,:,:] * units('m/s'))
vwnd = np.asarray(ds4.vwnd.sel(level=plevel).values[tm::12,:,:] * units('m/s'))



Print the numpy shapes of the temperature, latitude and longtitude. This can be important when debugging.

In [ ]:
print (np.shape(temp))
print (np.shape(lats))
print (np.shape(lons))
#print(temp[:,19,35])

Now we need to read in the ocen and land anomaly data. We read in from a csv file, which has the year and monthly values in columns. Note that the first column is year/month, followed by ocean anomaly and then land anomaly. 

In [ ]:
sourcefile = open('../data/landandocean.csv','rb')
landandocean = genfromtxt(sourcefile,delimiter=',')
print(np.shape(landandocean))
print(landandocean[0:5,:])
print(np.max(landandocean[1:1671,0]))

In [ ]:
plt.plot(landandocean[0:1671,0],landandocean[0:1671,2])
#plt.plot(landandocean[0:1671,0],landandocean[0:1671,1], 'bo')
plt.axis([188000, 201902, -1.6, 2.5])
plt.title('Land Anomaly')
plt.show()

In [ ]:
plt.plot(landandocean[0:1671,0],landandocean[0:1671,1])
#plt.plot(landandocean[0:1671,0],landandocean[0:1671,1], 'bo')
plt.axis([188000, 201902, -0.6, 0.9])
plt.title('Ocean Anomaly')
plt.show()

Subsetting January data since 1950 for ocean and land anomalies

In [ ]:
oceanJansince1950 = landandocean[1:1671:12,1][landandocean[1:1671:12,0]>=195000]
landJansince1950 = landandocean[1:1671:12,2][landandocean[1:1671:12,0]>=195000]
year = landandocean[1:1671:12,0][landandocean[1:1671:12,0]>=195000]
print(oceanJansince1950)
print(landJansince1950)
print(np.shape(landJansince1950))
print(year)

Now we calculate the correlation field between the index and the 500 hPa Z. The first step is to make objects with the right dimensions, before we can copy the correlation values. Note that the data is subset to leave off the first two years, for which the NAO data is not available. (The final year is left off too, since it is NA for some of the months.) 

In [ ]:
hgtcor=np.zeros((hght.shape[1],hght.shape[2]))
tempcor=np.zeros((temp.shape[1],temp.shape[2]))

for i in range(hght.shape[1]):
    for j in range(hght.shape[2]):
        tempcor[i,j] = np.corrcoef(temp[2:69,i,j],oceanJansince1950[0:67])[1,0]


Select the countour settings for the correlation The format is (minimum, maximum, interval) 

In [ ]:
clevs_cor = np.arange(-1,1.1,0.1)

In [ ]:
# Set up the projection that will be used for plotting
mapcrs = ccrs.Robinson(central_longitude=180)

# Set up the projection of the data; if lat/lon then PlateCarree is what you want
datacrs = ccrs.PlateCarree()

# Start the figure and create plot axes with proper projection
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111, projection=mapcrs)

#not needed here, but used for sub-setting domain to plot
#ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

# Add geopolitical boundaries for map reference
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
gl = ax.gridlines(color='gray',linewidth=2, alpha = 0.5, linestyle='--')

# labels gridlines for PlateCarree ccrs option (doesn't work with Robinson, etc.)
#gl.xlabels_top = False
#gl.ylabels_right = False

#color-filled contour time!
#plotting temperature and including color bar
cf = ax.contourf(lons, lats, tempcor, clevs_cor, cmap=plt.cm.RdYlBu_r, transform=datacrs)

plt.colorbar(cf, orientation='horizontal', pad=0.05, aspect=50)

# Plot 500-hPa Geopotential Height Anomaly in meters
#cs = ax.contour(lons, lats, hght[ty,:,:] - np.mean(hght[:,:,:],axis=0), clevs_hght_anom, colors='black', transform=datacrs)
#plt.clabel(cs, fmt='%d')

# Plot 500-hPa Geopotential Heights in meters
cs = ax.contour(lons, lats, tempcor, clevs_cor, colors='black', transform=datacrs)
plt.clabel(cs, fmt='%1.1f')

# Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
#ax.barbs(lons[::4], lats[::4], uwnd.to('kt')[ty,::4,::4].m, 
#         vwnd.to('kt')[ty,::4,::4].m, pivot='middle',
#         color='black', transform=datacrs, length=5, linewidth=0.4)

# Make some nice titles for the plot (one right, one left)
plt.title('Ocean Anomaly and 850 hPa Temperature Correlation Map ' + str(monthlabel))

# Reduce excessive white space around image and show
plt.savefig('oceananomaly500hPaZcorrelationmap' + str(monthlabel), dpi=150, bbox_inches='tight')
plt.show()

Now we calculate the correlation of the year with Reanalysis variables, to better understand trends

In [ ]:
hgtcor=np.zeros((hght.shape[1],hght.shape[2]))
tempcor=np.zeros((temp.shape[1],temp.shape[2]))
vwndcor=np.zeros((temp.shape[1],temp.shape[2]))

for i in range(hght.shape[1]):
    for j in range(hght.shape[2]):
        hgtcor[i,j] = np.corrcoef(hght[2:69,i,j], year[2:69])[1,0]
        tempcor[i,j] = np.corrcoef(temp[2:69,i,j],year[2:69])[1,0]
        vwndcor[i,j] = np.corrcoef(vwnd[2:69,i,j],year[2:69])[1,0]


In [ ]:
# Set up the projection that will be used for plotting
mapcrs = ccrs.Robinson(central_longitude=180)

# Set up the projection of the data; if lat/lon then PlateCarree is what you want
datacrs = ccrs.PlateCarree()

# Start the figure and create plot axes with proper projection
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111, projection=mapcrs)

#not needed here, but used for sub-setting domain to plot
#ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

# Add geopolitical boundaries for map reference
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
gl = ax.gridlines(color='gray',linewidth=2, alpha = 0.5, linestyle='--')

# labels gridlines for PlateCarree ccrs option (doesn't work with Robinson, etc.)
#gl.xlabels_top = False
#gl.ylabels_right = False

#color-filled contour time!
#plotting temperature and including color bar
cf = ax.contourf(lons, lats, vwndcor, clevs_cor, cmap=plt.cm.RdYlBu_r, transform=datacrs)

plt.colorbar(cf, orientation='horizontal', pad=0.05, aspect=50)

# Plot 500-hPa Geopotential Height Anomaly in meters
#cs = ax.contour(lons, lats, hght[ty,:,:] - np.mean(hght[:,:,:],axis=0), clevs_hght_anom, colors='black', transform=datacrs)
#plt.clabel(cs, fmt='%d')

# Plot 500-hPa Geopotential Heights in meters
cs = ax.contour(lons, lats, vwndcor, clevs_cor, colors='black', transform=datacrs)
plt.clabel(cs, fmt='%1.1f')

# Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
#ax.barbs(lons[::4], lats[::4], uwnd.to('kt')[ty,::4,::4].m, 
#         vwnd.to('kt')[ty,::4,::4].m, pivot='middle',
#         color='black', transform=datacrs, length=5, linewidth=0.4)

# Make some nice titles for the plot (one right, one left)
plt.title('Year and 200 hPa V Correlation Map ' + str(monthlabel))

# Reduce excessive white space around image and show
plt.savefig('yearand200hPaVcorrelationmap' + str(monthlabel), dpi=150, bbox_inches='tight')
plt.show()